<a href="https://colab.research.google.com/github/ma-prietoo/SyS-2025-2/blob/main/Taller2/Dashboard_1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Instalación de librerías
!pip install streamlit soundfile yt-dlp numpy matplotlib -q
!apt-get install ffmpeg -y -qq

# Instalar Cloudflare Tunnel
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -q
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# Crear carpeta pages para trabajar Multiapp en Streamlit
!rm -rf pages
!mkdir pages


In [14]:
# Página principal

%%writefile app.py
import streamlit as st

st.set_page_config(page_title="Dashboard")

st.title("Bienvenido")

st.markdown("""
Este dashboard contiene dos secciones principales:

### **Punto 1.6 Modulación AM**
Simulación completa de:
- Descarga de audio desde YouTube
- Extracción de un fragmento
- Modulación AM
- Visualización en tiempo y frecuencia
- Demodulación coherente

### **Punto 1.7 Potencia en Circuitos Eléctricos**
Simulación completa de:
- Rectificación de onda completa
- Carga resistiva
- Carga RC (filtro)
- Distorsión total de armónicos (THD)
- Factor de potencia basado en THD

Usa el menú lateral de Streamlit para navegar entre las secciones.
""")


Overwriting app.py


In [15]:
%%writefile pages/1_Modulación_AM.py
import streamlit as st
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import os
import shlex
import subprocess

st.title("Punto 1.6 Modulación AM")
st.markdown("""
**Ejercicio: Aplicación en comunicaciones**
Sea la señal portadora:
""")

st.latex(r"c(t) = A_c \cos(2\pi F_c t)")

st.markdown("""
Con $A_c, F_c \in \mathbb{R}$, y la señal mensaje $m(t) \in \mathbb{R}$.
Encuentre el espectro en frecuencia de la señal modulada en amplitud (AM):
""")

st.latex(r"y(t) = \left( 1 + \frac{m(t)}{A_c} \right) c(t)")

st.markdown("""
Luego, descargue desde YouTube 5 segundos de su canción favorita
(capturando del segundo 20 al 25).

Presente una simulación de modulación por amplitud AM
(tomando como mensaje el fragmento de la canción escogida
y con un índice de modulación de 1).

Grafique las señales en tiempo y frecuencia (magnitud) de:
- la señal mensaje,
- la señal portadora,
- y la señal modulada.

Reproduzca los fragmentos de audio del mensaje, portadora y señal modulada.

Nota: se sugiere utilizar un canal de señal de audio para el desarrollo del ejercicio.
Ver cuaderno guía de modulación AM.

Luego, sea el demodulador en amplitud presentado en la figura.
""")

st.markdown("""
Asumiendo θ0 = 0, determine el espectro de Fourier (teorico) en cada una de las etapas del sistema. Luego, con base en la simulacion de modulación en amplitud del Taller 2 y utilizando cinco segundos de una cancion de Youtube como mensaje, grafique cada una de las etapas principales del proceso de modulacion y demodulación en el tiempo y la frecuencia (reproduzca el segmento de la cancion en cada etapa).
**Nota:** Para la etapa de filtrado pasa bajas, realice su implementacion a partir de la transformada rápida de Fourier.
""")

# Teoría
st.header("Teoría")
st.markdown("""
En este ejercicio se estudia cómo una señal de audio \(m(t)\) puede transmitirse usando
**modulación en amplitud (AM)**.
El objetivo es "montar" el mensaje sobre una señal de alta frecuencia llamada **portadora**,
permitiendo su transmisión por un canal de comunicación.

La señal recibida del sistema DSB-CS se expresa como:
""")

st.latex(r"A_c\,m(t)\cos(2\pi f_c t + \theta_0)")

st.markdown("""
---

### Etapa 1: Mezclador (Multiplicación)

En el demodulador, la señal recibida se multiplica por una réplica de la portadora:
""")

st.latex(r"\cos(2\pi f_c t + \theta_0)")

st.markdown("""
La salida del mezclador se obtiene multiplicando ambas señales:
""")

st.latex(r"A_c m(t)\cos(2\pi f_c t+\theta_0)\cos(2\pi f_c t+\theta_0)")

st.markdown("""
Se usa la identidad trigonométrica fundamental:
""")

st.latex(r"\cos^2(x)=\frac{1}{2}+\frac{1}{2}\cos(2x)")

st.markdown("""
Aplicando la identidad, la salida del mezclador se descompone en:
""")

st.latex(r"""
A_c m(t)\cos^2(2\pi f_c t+\theta_0)
= \frac{A_c m(t)}{2}
+ \frac{A_c m(t)}{2}\cos(4\pi f_c t + 2\theta_0)
""")

st.markdown("""
---

### Etapa 2: Filtro Pasa Bajas (LPF)

El filtro elimina la componente de alta frecuencia:
""")

st.latex(r"\frac{A_c m(t)}{2}\cos(4\pi f_c t + 2\theta_0)")

st.markdown("Quedando solamente:")

st.latex(r"\frac{A_c}{2}\, m(t)")

st.markdown("""
---

### Etapa 3: Normalización

Para recuperar el mensaje original, se aplica el factor:
""")

st.latex(r"\frac{2}{A_c}")

st.markdown("Obteniendo finalmente:")

st.latex(r"m(t)=\frac{2}{A_c}\left( \frac{A_c}{2}m(t) \right)")

st.markdown("""
---

Este proceso permite analizar cómo cambia el espectro del mensaje a lo largo de cada etapa del
sistema: la señal original, la portadora, la señal AM, la salida del mezclador, la señal filtrada
y el mensaje recuperado.
""")

# Link de YouTube
st.header("Descargar Audio desde YouTube")

default_link = "https://www.youtube.com/watch?v=YglZ5BzPbYs&list=RDYglZ5BzPbYs&start_radio=1"
link = st.text_input("Ingresa link de YouTube:", value=default_link) # Enlace por defecto

if st.button("Descargar Audio"):

    with st.spinner("Descargando audio…"):
        # Eliminar archivos previos (si existen)
        for fn in ("audio.mp3", "audio.m4a", "audio.webm", "output.wav", "audio"):
            try:
                os.remove(fn)
            except Exception:
                pass

        # Construir comando de descarga con salida fija (extensible)
        # Se fuerza nombre audio.<ext> para luego convertir a wav
        cmd = f'yt-dlp --extract-audio -o "audio.%(ext)s" --audio-format mp3 --force-overwrites "{link}"'
        try:
            subprocess.run(shlex.split(cmd), check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except subprocess.CalledProcessError as e:
            st.error("Error al descargar el audio con yt-dlp. Revisa el enlace o la instalación de yt-dlp.")
            st.code(e.stderr.decode("utf-8")[:1000])
            st.stop()

        # Detectar archivo descargado (buscar audio)
        downloaded = None
        for ext in ("mp3", "m4a", "webm", "wav"):
            candidate = f"audio.{ext}"
            if os.path.exists(candidate):
                downloaded = candidate
                break

        if downloaded is None:
            st.error("No se encontró el archivo descargado.")
            st.stop()

        # Convertir a WAV con ffmpeg (si no es wav)
        if not downloaded.lower().endswith(".wav"):
            cmd_conv = f'ffmpeg -y -i "{downloaded}" output.wav'
            try:
                subprocess.run(shlex.split(cmd_conv), check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                wav_path = "output.wav"
            except subprocess.CalledProcessError as e:
                st.error("Error al convertir a WAV con ffmpeg.")
                st.code(e.stderr.decode("utf-8")[:1000])
                st.stop()
        else:
            wav_path = downloaded

    st.success("Audio descargado y convertido")

    # Leer y extraer fragmento 20–25 s
    try:
        audio, fs = sf.read(wav_path)
    except Exception as e:
        st.error(f"No se pudo leer el archivo WAV: {e}")
        st.stop()

    if audio.ndim > 1:  # Si es estéreo, convertir a mono
        audio = np.mean(audio, axis=1)

    ti, tf = 20, 25 # Segmento de 20 a 25 segundos
    start = int(ti * fs)
    end = int(tf * fs)
    if end > len(audio):
        st.warning("El archivo es más corto que 25 s; se usará el tramo disponible al final.")
        start = max(0, len(audio) - int(5 * fs))
        end = len(audio)

    segmento = audio[start:end]
    if np.max(np.abs(segmento)) == 0:
        st.error("El segmento seleccionado es todo ceros.")
        st.stop()

    mensaje = segmento / np.max(np.abs(segmento)) # Normalización
    t_audio = np.arange(len(mensaje)) / fs # Tiempo para el gráfico

    # Reproducir el fragmento de audio
    st.subheader("Fragmento (20–25 s)")
    st.audio(mensaje, sample_rate=fs)

    # Modulación AM
    st.header("Modulación AM")

    # Parámetros para la modulación AM
    Fc = 5000  # Frecuencia de la portadora (5000 Hz)
    Ac = 1     # Amplitud de la portadora
    Im = 1     # Índice de modulación

    # Señal portadora c(t)
    portadora = Ac * np.cos(2 * np.pi * Fc * t_audio)
    # Señal modulada en AM: y(t) = (1 + m(t)) * Ac * cos(2πFc t)
    # Nota: se mantiene Im para ajustar índice si se desea
    y = (1 + Im * mensaje) * portadora

    # Normalizar y para evitar clipping al reproducir
    y = y / (np.max(np.abs(y)) + 1e-12)

    # Reproducir las señales
    st.audio(y, sample_rate=fs)

    # Función para calcular espectro de frecuencia usando FFT
    def espectro(x, Fs, N_fft=2**15):
        """
        Devuelve frecuencias y magnitud normalizada del espectro (valores positivos).
        """
        # Ventana para reducir fugas
        win = np.hanning(len(x))
        xw = x * win
        # Zero-pad o recortar a N_fft
        X = np.fft.rfft(xw, n=N_fft)
        f = np.fft.rfftfreq(N_fft, 1 / Fs)
        # Convertir a escala de magnitud adecuada (2/N para conservar energía en rfft)
        mag = (2.0 / np.sum(win)) * np.abs(X)
        return f, mag

    # Calcular espectros
    f_m, M = espectro(mensaje, fs)
    f_y, Y = espectro(y, fs)

    # Graficar señales en el tiempo
    st.subheader("Señales en el Tiempo")
    fig, ax = plt.subplots(4, 1, figsize=(10, 9), sharex=True)
    ax[0].plot(t_audio, mensaje); ax[0].set_title("Mensaje"); ax[0].grid()
    ax[1].plot(t_audio, portadora, color='orange'); ax[1].set_title("Portadora"); ax[1].grid()
    ax[2].plot(t_audio, y, color='green'); ax[2].set_title("Señal AM"); ax[2].grid()

    # Etapa de Mixer DSB-CS: Mezcla del mensaje con la portadora (mensaje * cos(2πFc t))
    # (mostramos la señal al mezclar para visualizar contenid o de banda lateral)
    mixer = mensaje * np.cos(2 * np.pi * Fc * t_audio)
    ax[3].plot(t_audio, mixer, color='purple'); ax[3].set_title("Señal después del Mixer DSB-CS"); ax[3].grid()

    # Ajustar el espacio entre subgráficos (hspace controla el espacio vertical)
    plt.subplots_adjust(hspace=0.6)

    st.pyplot(fig)

    # Graficar espectros de frecuencia
    st.subheader("Espectros de Frecuencia")
    fig2, ax2 = plt.subplots(3, 1, figsize=(10, 10))

    # Espectro del mensaje
    ax2[0].plot(f_m, M, label="Mensaje")
    ax2[0].set_xlim(0, min(12000, fs/2))
    ax2[0].set_title("Espectro del Mensaje")
    ax2[0].grid()

    # Espectro de la señal AM
    ax2[1].plot(f_y, Y, label="Señal AM")
    ax2[1].set_xlim(0, min(12000, fs/2))
    ax2[1].set_title("Espectro de la Señal AM")
    ax2[1].grid()

    # Espectro después del Mixer DSB-CS
    f_mixer, M_mixer = espectro(mixer, fs)
    ax2[2].plot(f_mixer, M_mixer, label="Después del Mixer DSB-CS")
    ax2[2].set_xlim(0, min(12000, fs/2))
    ax2[2].set_title("Espectro después del Mixer DSB-CS")
    ax2[2].grid()

    plt.subplots_adjust(hspace=0.6)

    st.pyplot(fig2)

    # Demodulación
    st.header("Demodulación")

    # Multiplicación de la señal modulada por la portadora local (coherente)
    demodulada = y * portadora

    # FFT de la señal demodulada
    N_dem = len(demodulada)
    X = np.fft.rfft(demodulada)
    freqs = np.fft.rfftfreq(N_dem, 1 / fs)

    # Filtro pasa-bajas
    cutoff = min(8000, fs / 2 - 1)
    X_f = X.copy()
    X_f[freqs > cutoff] = 0  # Frecuencia de corte del LPF

    # Reconstruir señal en tiempo
    mensaje_rec = np.fft.irfft(X_f, n=N_dem)

    # Compensación por la multiplicación por la portadora (factor 1/2) y por ventana si aplica
    # (en nuestro caso, hacemos una normalización sencilla)
    if np.max(np.abs(mensaje_rec)) > 0:
        mensaje_rec = mensaje_rec / np.max(np.abs(mensaje_rec))
    else:
        st.warning("Señal recuperada es prácticamente cero después del filtrado.")

    # Recortar/pad si por alguna razón la longitud difiere (asegurar misma longitud para graficar)
    L = min(len(mensaje), len(mensaje_rec))
    mensaje_rec = mensaje_rec[:L]
    mensaje_plot = mensaje[:L]
    t_plot = t_audio[:L]

    # Reproducir mensaje recuperado
    st.audio(mensaje_rec, sample_rate=fs)

    # Graficar mensaje original vs recuperado
    fig3, ax3 = plt.subplots(figsize=(10, 4))
    ax3.plot(t_plot, mensaje_plot, '--', label="Original")
    ax3.plot(t_plot, mensaje_rec, color='red', label="Recuperado")
    ax3.set_title("Mensaje original vs Recuperado")
    ax3.grid()
    ax3.legend()
    st.pyplot(fig3)

    st.success("Demodulación completada.")


Writing pages/1_Modulación_AM.py


In [16]:
%%writefile pages/2_Potencia_en_Circuitos_Eléctricos.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

st.set_page_config(page_title="THD y Potencia", layout="wide")

# Título y teoría (tomada del archivo Punto_1_7.txt)
st.title("Punto 1.7 — Distorsión Armónica (THD) y Factor de Potencia (PF)")

st.markdown("""
**Ejercicio 1: Aplicación en circuitos eléctricos - potencia**

Consulte en qué consiste la distorsión total de armónicos (Total Harmonic Distortion — THD) y el factor de potencia en un circuito eléctrico. Cómo puede calcularse el THD desde la FFT y cómo puede calcularse la distorsión del factor de potencia con base al THD.
""")

st.header("¿Qué es el THD?")

st.markdown("""
La Distorsión Total de Armónicos (Total Harmonic Distortion, THD) mide la proporción de energía presente
en las componentes armónicas (segundo, tercero, ...) respecto a la componente fundamental de una señal periódica.
Si $V_n$ es el valor RMS (o magnitud eficaz) del $n$-ésimo armónico y $V_1$ el RMS del armónico fundamental,
el THD se define como:
""")

st.latex(r"\text{THD} = \frac{\sqrt{V_2^2 + V_3^2 + \dots + V_N^2}}{V_1}")

st.markdown("""
A veces se expresa en porcentaje: $\\text{THD}\\% = 100 \cdot \\text{THD}$.
""")

st.header("Factor de potencia (PF) en un circuito eléctrico")

st.markdown("""
Mide qué tan eficientemente un circuito usa la energía eléctrica. Indica la proporción entre la potencia útil
(activa) y la potencia total suministrada.

**En circuitos senoidales puros:**
Si la corriente y el voltaje son senoidales, el factor de potencia se calcula como:
""")

st.latex(r"PF = \cos(\varphi)")

st.markdown("""
donde $\\varphi$ es el ángulo de desfase entre el voltaje y la corriente.

Si $\\varphi = 0$, entonces $PF = 1$ (máxima eficiencia).
Si $\\varphi > 0$, aparece energía reactiva y $PF < 1$.

**En circuitos no senoidales (con distorsión):**
Cuando la señal tiene armónicos, el factor de potencia también disminuye aunque no haya desfase.
En este caso se usa la relación con la distorsión armónica total (THD):
""")

st.latex(r"THD = \frac{\sqrt{V_2^2 + V_3^2 + \cdots + V_n^2}}{V_1}")

st.latex(r"PF_{\text{THD}} = \frac{1}{\sqrt{1 + THD^2}}")

st.markdown("donde $V_1$ es la componente fundamental y $V_2, V_3, \\dots$ son los armónicos.")

st.header("¿Cómo se calcula el THD desde la FFT?")

st.markdown("""
1. Calcular la FFT (o rFFT para señales reales) de la señal periódica muestreada.
2. Obtener el espectro single-sided (solo frecuencias positivas) y normalizarlo (dividir por el número de muestras).
3. Identificar el bin correspondiente a la frecuencia fundamental $f_1$ y sus múltiplos (armónicos).
4. Obtener $V_n$ como la magnitud eficaz de cada armónico (conversión pico → RMS si es necesario).
5. Aplicar la fórmula del THD.

**Filtro práctico para identificar armónicos:**
Buscar picos en las frecuencias próximas a $k f_1$, con $k = 1, 2, \dots$.
""")

st.header("¿Cómo puede calcularse la distorsión del factor de potencia con base al THD?")

st.markdown("""
Si una señal posee armónicos, la componente armónica reduce la potencia útil asociada a la fundamental.
Una aproximación simple que relaciona THD con la pérdida de factor de potencia por distorsión es:
""")

st.latex(r"\text{PF}_{\text{THD}} \approx \frac{1}{\sqrt{1 + \text{THD}^2}}")

st.markdown("""
Esta expresión supone que el desfase de la componente fundamental es nulo (o despreciable) y que el efecto principal
es la presencia de armónicos.
""")

st.markdown("---")
st.markdown("""
**Ejercicio 2: Rectificador de onda completa (carga R y carga RC)**

Genere un ejemplo ilustrativo para el cálculo del THD y la distorsión del factor de potencia para un rectificador de onda completa con carga:
i) netamente resistiva y ii) carga RC en serie.
Establezca las condiciones necesarias para las simulaciones y pruebe con diferentes valores de R y C. Discuta los resultados obtenidos.
""")

# Sidebar - parámetros
st.sidebar.header("Parámetros de simulación")
A = st.sidebar.slider("Amplitud de la fuente A [V]", 10, 240, 120)
Fo = st.sidebar.slider("Frecuencia de la fuente Fo [Hz]", 10, 120, 60)
num_periods = st.sidebar.slider("Número de periodos a simular", 1, 20, 5)
Fs = st.sidebar.slider("Frecuencia de muestreo Fs [Hz]", 1000, 200000, 30*60)  # pred: 30*Fo por defecto
R_default = 1000
R = st.sidebar.number_input("Resistencia R [Ω]", min_value=1.0, value=float(R_default))
use_cap = st.sidebar.checkbox("Simular carga RC (agregar C)", value=True)
C = None
if use_cap:
    C = st.sidebar.number_input("Capacitancia C [F]", min_value=1e-9, max_value=1.0, value=10e-6, format="%.8f")

st.sidebar.markdown("---")
st.sidebar.write("Control de visualización")
show_time = st.sidebar.checkbox("Mostrar gráfica en tiempo", True)
show_spectrum = st.sidebar.checkbox("Mostrar espectros (FFT)", True)
show_signals_separately = st.sidebar.checkbox("Mostrar señales por separado", False)

# Señal temporal
To = 1.0 / Fo
Ts = 1.0 / Fs
t = np.arange(0, num_periods * To, Ts)

# Entrada senoidal y rectificada (onda completa)
in_o = A * np.sin(2 * np.pi * Fo * t)
rec_c = sig.square(2 * np.pi * Fo * t)  # genera ±1; con producto obtenemos onda rectificada (full-wave)
in_r = in_o * rec_c

# Función que simula la salida del "filtro" carga (R o RC serie)
def salida_rectificador(R_v, C_v=None, input_signal=None, tvec=None):
    if input_signal is None:
        input_signal = in_r
    if tvec is None:
        tvec = t
    if C_v is None:
        # carga puramente resistiva: salida = entrada (modelo ideal de carga)
        return input_signal.copy()
    else:
        # modelo RC serie como un filtrado (simple primer orden)
        # Transfer function: 1 / (R*C*s + 1)
        num = [1.0]
        den = [R_v * C_v, 1.0]
        G = sig.TransferFunction(num, den)
        # usar lsim (o .output si disponible). En scipy actual: sig.lsim(G, U, T)
        try:
            tout, yout, xout = sig.lsim(G, U=input_signal, T=tvec)
            return yout
        except Exception:
            # Si lsim no disponible en tu versión, aplicar respuesta por filtro exponencial (approx)
            alpha = Ts / (R_v * C_v + Ts)
            y = np.zeros_like(input_signal)
            for n in range(1, len(input_signal)):
                y[n] = y[n-1] + alpha * (input_signal[n] - y[n-1])
            return y

# Cálculo de FFT, THD y PF
def calc_THD_PF(signal, Fs_local):
    N = len(signal)
    # rFFT y frecuencias
    freqs = np.fft.rfftfreq(N, d=1.0/Fs_local)
    spectrum = np.abs(np.fft.rfft(signal)) / N

    # identificar fundamental (pico más fuerte)
    fundamental_bin = np.argmax(spectrum[1:]) + 1 if len(spectrum) > 1 else 0
    V1 = spectrum[fundamental_bin] if fundamental_bin < len(spectrum) else 1e-12

    # ignorar DC (bin 0) al calcular armónicos si se desea
    harmonics = np.copy(spectrum)
    harmonics[fundamental_bin] = 0.0
    # opcional: eliminar DC del término armónico
    harmonics[0] = 0.0

    THD = np.sqrt(np.sum(harmonics**2)) / (V1 + 1e-16)
    PF_THD = 1.0 / np.sqrt(1.0 + THD**2)
    return THD, PF_THD, freqs, spectrum, fundamental_bin

# Generar salidas
out_R = salida_rectificador(R, C_v=None, input_signal=in_r, tvec=t)
if use_cap:
    out_RC = salida_rectificador(R, C_v=C, input_signal=in_r, tvec=t)
else:
    out_RC = None

# Calcular métricas
THD_R, PF_R, fR, specR, fund_bin_R = calc_THD_PF(out_R, Fs)
if use_cap:
    THD_RC, PF_RC, fRC, specRC, fund_bin_RC = calc_THD_PF(out_RC, Fs)

# Mostrar resultados numéricos
st.header("Resultados numéricos")
col1, col2, col3 = st.columns(3)
col1.metric("THD — Carga R", f"{THD_R:.4f}", f"{THD_R*100:.2f}%")
col2.metric("Factor de Potencia (por THD) — R", f"{PF_R:.4f}")
if use_cap:
    col3.metric("THD — Carga RC", f"{THD_RC:.4f}", f"{THD_RC*100:.2f}%")

if use_cap:
    st.write("")  # separador visual menor
    col4, col5 = st.columns(2)
    col4.metric("Factor de Potencia (por THD) — RC", f"{PF_RC:.4f}")
    col5.write(f"Fundamental R bin: {fund_bin_R}, Fundamental RC bin: {fund_bin_RC}")

# Gráficas en tiempo
if show_time:
    st.header("Señales en el tiempo")
    fig_t, ax_t = plt.subplots(figsize=(10, 4))
    ax_t.plot(t, in_r, label="Entrada rectificada", linewidth=2, alpha=0.6)
    ax_t.plot(t, out_R, label="Salida (R)", linewidth=1.5)
    if use_cap and out_RC is not None:
        ax_t.plot(t, out_RC, label="Salida (RC)", linewidth=1.5)
    ax_t.set_xlabel("t [s]")
    ax_t.set_ylabel("Amplitud")
    ax_t.set_title("Rectificador con diferentes cargas")
    ax_t.grid(True, alpha=0.3)
    ax_t.legend()
    st.pyplot(fig_t)

# Mostrar cada señal por separado si el usuario lo desea
if show_signals_separately:
    st.subheader("Señales individuales")
    st.markdown("**Entrada rectificada**")
    fig_in, ax_in = plt.subplots(figsize=(10,2))
    ax_in.plot(t, in_r)
    ax_in.set_xlim(t[0], t[-1])
    st.pyplot(fig_in)

    st.markdown("**Salida carga R**")
    fig_r, ax_r = plt.subplots(figsize=(10,2))
    ax_r.plot(t, out_R)
    ax_r.set_xlim(t[0], t[-1])
    st.pyplot(fig_r)

    if use_cap:
        st.markdown("**Salida carga RC**")
        fig_rc, ax_rc = plt.subplots(figsize=(10,2))
        ax_rc.plot(t, out_RC)
        ax_rc.set_xlim(t[0], t[-1])
        st.pyplot(fig_rc)

# Espectros (FFT)
if show_spectrum:
    st.header("Espectros (FFT) — Magnitud single-sided")
    fig_f, ax_f = plt.subplots(figsize=(10,4))
    ax_f.plot(fR, specR, label="Carga R")
    if use_cap:
        ax_f.plot(fRC, specRC, label="Carga RC")
    ax_f.set_xlim(0, min(Fs/2, 2000))
    ax_f.set_xlabel("Frecuencia [Hz]")
    ax_f.set_ylabel("|X(f)|")
    ax_f.set_title("Espectro de salida (magnitud)")
    ax_f.grid(True)
    ax_f.legend()
    st.pyplot(fig_f)

st.markdown("---")
st.header("Interpretación de resultados")

st.markdown("""

1. **Carga resistiva pura (R):** Cuando la carga del rectificador es puramente resistiva, la corriente y el voltaje están en fase. En este caso:

  * La señal de salida tiene la misma forma que la señal rectificada de entrada, con pulsos positivos que siguen directamente el semiciclo positivo de la entrada.
  * No hay almacenamiento de energía, por lo que el voltaje cae a cero entre cada pico.
  * El contenido armónico de la señal es elevado, ya que la onda rectificada se compone de muchos armónicos impares y pares que surgen de la no linealidad del rectificador.
  * Por ello, el THD (Total Harmonic Distortion) resulta alto, indicando una gran cantidad de distorsión respecto al fundamental.
  * Sin embargo, como no hay desfase entre voltaje y corriente, el factor de potencia (PF) sigue siendo alto (cercano a 1).

    **Conclusión:** La carga resistiva produce una señal con alta distorsión armónica, pero mantiene un buen factor de potencia al no introducir desfases.

2. **Carga RC (resistencia + capacitor):** Cuando se conecta un capacitor en paralelo con la resistencia, el comportamiento del circuito cambia considerablemente:

* El capacitor se carga durante los picos de la onda rectificada y se descarga lentamente entre ciclos, lo que genera una salida más suave o filtrada (menos ondulaciones o rizado).
* Esto significa que el THD disminuye, ya que el contenido de armónicos se reduce al tener una señal más continua.
* Sin embargo, el capacitor introduce un desfase entre el voltaje y la corriente (la corriente se adelanta al voltaje), afectando negativamente el factor de potencia.
* Por tanto, aunque el THD mejora, el PF disminuye debido al carácter reactivo del circuito.

**Conclusión:** La carga RC suaviza la forma de onda y reduce la distorsión armónica, pero introduce un desfase que reduce el factor de potencia total del sistema.
""")




Writing pages/2_Potencia_en_Circuitos_Eléctricos.py


In [17]:
# Ejecutar Streamlit
!streamlit run app.py &>/content/logs.txt &

# Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

# Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL


Tu aplicación está disponible en: https://warranties-ferrari-fwd-husband.trycloudflare.com


In [18]:
import os

res = input("Digite (0) para finalizar la ejecución del Dashboard: ")

if res.upper() == "0":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")

Digite (0) para finalizar la ejecución del Dashboard: 0
El proceso de Streamlit ha sido finalizado.
